# Session 4 — Paragraph-Level Analysis
## Measure 5: Paragraph Function Classification

In this notebook, you will:
- define a small set of **paragraph function labels**
  (dialogue, action, description, internal_monologue, other)
- create short **prototype texts** for each label
- embed both prototypes and real paragraphs with MiniLM
- assign to each paragraph the label whose prototype is most similar.

This mimics how agentic LLM systems classify chunks of text into roles
(instruction, context, plan, reflection, etc.).


In [ ]:
import re
from typing import List, Dict
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

# You may need to install this once in your environment:
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def load_book(filepath: str) -> str:
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    if 'CHAPTER I' in text:
        start = text.find('CHAPTER I')
        text = text[start:]
    elif '*** START OF' in text:
        start = text.find('*** START OF')
        text = text[start + 100:]

    if '*** END OF' in text:
        end = text.find('*** END OF')
        text = text[:end]
    elif 'End of Project Gutenberg' in text:
        end = text.find('End of Project Gutenberg')
        text = text[:end]

    return text.strip()

wonderland_text = load_book('../data/Wonderland.txt')
looking_glass_text = load_book('../data/Looking-Glass.txt')

print(f"Wonderland characters: {len(wonderland_text):,}")
print(f"Looking-Glass characters: {len(looking_glass_text):,}")


In [ ]:
def split_into_paragraphs(text: str, min_words: int = 10) -> List[str]:
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    raw_paras = re.split(r'\n\s*\n+', text)
    paras = []
    for p in raw_paras:
        cleaned = re.sub(r'\s+', ' ', p).strip()
        if not cleaned:
            continue
        if len(cleaned.split()) < min_words:
            continue
        paras.append(cleaned)
    return paras

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    if a.ndim > 1:
        a = a.reshape(-1)
    if b.ndim > 1:
        b = b.reshape(-1)
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

# Define prototype texts for each label.
# You can edit these to better fit your author's style.
PROTOTYPES: Dict[str, List[str]] = {
    'dialogue': [
        '"What are you doing here?" she asked quietly.',
        '"I cannot explain it," he replied, "but I must try."',
    ],
    'action': [
        'She ran down the hallway and opened the heavy door.',
        'He turned quickly, jumped over the fence, and disappeared into the forest.',
    ],
    'description': [
        'The room was small and dimly lit, with dusty books lining the wooden shelves.',
        'A cold wind moved softly through the tall, dark trees of the silent garden.',
    ],
    'internal_monologue': [
        'I wondered whether I had made a terrible mistake and what would happen next.',
        'In my mind I repeated the same question again and again, unable to decide.',
    ],
    'other': [
        'The narrator provides context and explanation that does not fit clearly into the other categories.',
    ],
}

def build_prototype_embeddings() -> Dict[str, np.ndarray]:
    proto_embs = {}
    for label, examples in PROTOTYPES.items():
        embs = model.encode(examples)
        proto_embs[label] = embs.mean(axis=0)
    return proto_embs

prototype_embeddings = build_prototype_embeddings()

def classify_paragraph_embedding(paragraph: str, proto_embs: Dict[str, np.ndarray]) -> str:
    """Assign the label whose prototype embedding is most similar."""
    emb = model.encode(paragraph)
    best_label = None
    best_sim = -1.0
    for label, proto in proto_embs.items():
        sim = cosine_similarity(emb, proto)
        if sim > best_sim:
            best_sim = sim
            best_label = label
    return best_label or 'other'

def classify_all(paragraphs: List[str], proto_embs: Dict[str, np.ndarray]) -> List[str]:
    labels = []
    for p in paragraphs:
        if len(p.split()) < 5:
            continue
        labels.append(classify_paragraph_embedding(p, proto_embs))
    return labels

wonderland_paras = split_into_paragraphs(wonderland_text)
looking_glass_paras = split_into_paragraphs(looking_glass_text)

w_labels = classify_all(wonderland_paras, prototype_embeddings)
g_labels = classify_all(looking_glass_paras, prototype_embeddings)

def label_counts(labels: List[str]) -> Counter:
    c = Counter(labels)
    total = sum(c.values()) or 1
    for k, v in c.items():
        print(f"{k:18s}: {v:4d} ({v/total*100:5.1f}%)")
    return c

print("Wonderland paragraph types (embedding-based):")
w_counts = label_counts(w_labels)

print("\nLooking-Glass paragraph types (embedding-based):")
g_counts = label_counts(g_labels)


In [ ]:
def plot_label_bars(counts: Counter, title: str, ax):
    labels = list(counts.keys())
    values = [counts[k] for k in labels]
    ax.bar(labels, values)
    ax.set_title(title)
    ax.set_ylabel('Number of paragraphs')
    ax.grid(True, axis='y', alpha=0.3)

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

plot_label_bars(w_counts, 'Wonderland paragraph functions (MiniLM)', axes[0])
plot_label_bars(g_counts, 'Looking-Glass paragraph functions (MiniLM)', axes[1])

plt.tight_layout()
plt.show()


In [ ]:
def show_examples(paragraphs: List[str], labels: List[str], target_label: str, n: int = 3):
    print(f"\n=== Examples for label: {target_label} ===")
    count = 0
    for p, lab in zip(paragraphs, labels):
        if lab == target_label:
            print('\n---')
            snippet = p.strip()
            if len(snippet) > 500:
                snippet = snippet[:500] + '...'
            print(snippet)
            count += 1
            if count >= n:
                break

show_examples(wonderland_paras, w_labels, 'dialogue')
show_examples(wonderland_paras, w_labels, 'action')
show_examples(wonderland_paras, w_labels, 'internal_monologue')


## Memory Cleanup

If you're running low on memory, run this cell to free up RAM by deleting large variables and clearing the model cache.

In [ ]:
import gc

# Delete large variables to free memory
del wonderland_text, looking_glass_text
del wonderland_paras, looking_glass_paras
del w_labels, g_labels, w_counts, g_counts
del prototype_embeddings

# Clear matplotlib figures
plt.close('all')

# Unload the model from memory
del model

# Force garbage collection
gc.collect()

print("Memory cleaned! Large variables deleted and garbage collected.")